### Install ngrok library

In [1]:
!pip install fastapi
!pip install uvicorn
!pip install requests
!pip install pypi-json
!pip install pyngrok
!pip install nest-asyncio

In [2]:
from fastapi import FastAPI
from pydantic import BaseModel
import pickle
import json
import uvicorn
from pyngrok import ngrok
from fastapi.middleware.cors import CORSMiddleware
import nest_asyncio

In [3]:
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel
import torch

from fastapi.middleware.cors import CORSMiddleware
from io import BytesIO
import base64
from PIL import Image

app = FastAPI()

# CORS configuration
origins = [
    "http://localhost:3000",  # Add the URL of your frontend app
    "http://127.0.0.1:3000",  # Localhost may also use this URL
    # "https://yourproductiondomain.com",  # Uncomment this line for your production domain if necessary
]

app.add_middleware(
    CORSMiddleware,
    allow_origins=origins,  # List of allowed origins
    allow_credentials=True,
    allow_methods=["*"],  # Allows all methods
    allow_headers=["*"],  # Allows all headers
)

# Text-to-image api endpoint

# hugging face login

In [4]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: fineGr

In [5]:
!pip install diffusers

In [6]:
import torch
from diffusers import StableDiffusionPipeline, DPMSolverMultistepScheduler

class stable_diffusion(BaseModel):
    prompt: str
    height: int
    width: int
    num_inference_steps: int
    guidance_scale: float
    num_images_per_prompt: int
    # timesteps: List[int]
    # sigmas: List[float]

model_id = "stabilityai/stable-diffusion-2-1"

# Use the DPMSolverMultistepScheduler (DPM-Solver++) scheduler here instead
pipe = StableDiffusionPipeline.from_pretrained(model_id, torch_dtype=torch.float16)
pipe.scheduler = DPMSolverMultistepScheduler.from_config(pipe.scheduler.config)
pipe = pipe.to("cuda")

def text_to_image(input):
    image = pipe(
        input.prompt,
        height = input.height,
        width = input.width,
        negative_prompt="",
        num_inference_steps=input.num_inference_steps,
        guidance_scale= input.guidance_scale,
        num_images_per_prompt=input.num_images_per_prompt
        # timesteps = input.timesteps,
        # sigmas = input.sigmas
    ).images[0]

    # Convert the image to a base64 string
    buffered = BytesIO()
    image.save(buffered, format="PNG")
    img_str = base64.b64encode(buffered.getvalue()).decode("utf-8")

    return img_str

@app.post("/api/text_to_image")
async def api_text_to_image(item: stable_diffusion):
    result = text_to_image(item)
    return {"output": result}


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading pipeline components...:   0%|          | 0/6 [00:00<?, ?it/s]

# Chattts API end point

In [7]:
!pip install ChatTTS
!pip install numpy

In [8]:
import torch
import ChatTTS
import torchaudio
from IPython.display import Audio
import numpy as np

class chatTTS(BaseModel):
    input: str
    temperature: float ## 0.00001 - 1
    top_P: float ## 0.1-0.9
    top_K: int ## 1- 20
    oral: int ##
    laugh: int ##
    break_: int ##
    seed: int

chat = ChatTTS.Chat()
chat.load(compile=False) # Set to True for better performance

def deterministic(seed=0):
    torch.manual_seed(seed)
    np.random.seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

def generate_audio_from_text(user_input):

    deterministic(user_input.seed)
    rand_spk = chat.sample_random_speaker()
    params_infer_code = ChatTTS.Chat.InferCodeParams(
      spk_emb = rand_spk, # add sampled speaker
      temperature = user_input.temperature,   # using custom temperature
      top_P = user_input.top_P,        # top P decode
      top_K = user_input.top_K,         # top K decode
    )

    ###################################
    # For sentence level manual control.

    # use oral_(0-9), laugh_(0-2), break_(0-7)
    # to generate special token in text to synthesize.
    params_refine_text = ChatTTS.Chat.RefineTextParams(
        prompt=f'[oral_{user_input.oral}][laugh_{user_input.laugh}][break_{user_input.break_}]',
    )

    wavs = chat.infer(
        user_input.input,
        params_refine_text=params_refine_text,
        params_infer_code=params_infer_code,
    )

    audio_file = BytesIO()
    torchaudio.save(audio_file, torch.tensor(wavs[0]), 24000, format="wav")
    audio_file.seek(0)
    audio_base64 = base64.b64encode(audio_file.read()).decode('utf-8')

    return audio_base64

@app.post("/api/generate_audio")
async def api_generate_audio(item: chatTTS):
    audio = generate_audio_from_text(item)
    return {"output": audio}

/usr/local/lib/python3.10/dist-packages/ChatTTS/norm.py:14: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def _find_index(table: np.ndarray, val: np.uint16):
/usr/local/lib/python3.10/dist-packages/ChatTTS/norm.py:22: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def _fast_replace(


# **Animated Diff API endpoint** Working on this rn

---



In [9]:
import torch
from diffusers import AnimateDiffPipeline, MotionAdapter, EulerDiscreteScheduler
from diffusers.utils import export_to_gif
from huggingface_hub import hf_hub_download
from safetensors.torch import load_file

device = "cuda"
dtype = torch.float16

step = 4  # Options: [1,2,4,8]
repo = "ByteDance/AnimateDiff-Lightning"
ckpt = f"animatediff_lightning_{step}step_diffusers.safetensors"
base = "emilianJR/epiCRealism"  # Choose to your favorite base model.

adapter = MotionAdapter().to(device, dtype)
adapter.load_state_dict(load_file(hf_hub_download(repo ,ckpt), device=device))
pipe_ = AnimateDiffPipeline.from_pretrained(base, motion_adapter=adapter, torch_dtype=dtype).to(device)
pipe_.scheduler = EulerDiscreteScheduler.from_config(pipe_.scheduler.config, timestep_spacing="trailing", beta_schedule="linear")


Loading pipeline components...:   0%|          | 0/6 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/clip/feature_extraction_clip.py:28: FutureWarning: The class CLIPFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use CLIPImageProcessor instead.
  warnings.warn(
The config attributes {'center_input_sample': False, 'flip_sin_to_cos': True, 'freq_shift': 0, 'mid_block_type': 'UNetMidBlock2DCrossAttn', 'only_cross_attention': False, 'attention_head_dim': 8, 'dual_cross_attention': False, 'class_embed_type': None, 'num_class_embeds': None, 'upcast_attention': False, 'resnet_time_scale_shift': 'default', 'resnet_skip_time_act': False, 'resnet_out_scale_factor': 1.0, 'time_embedding_type': 'positional', 'time_embedding_dim': None, 'time_embedding_act_fn': None, 'timestep_post_act': None, 'conv_in_kernel': 3, 'conv_out_kernel': 3, 'class_embeddings_concat': False, 'mid_block_only_cross_attention': None, 'cross_attention_norm': None, 'addition_embed_type_num_heads': 64} were passed to UNetMotio

In [10]:
class animate(BaseModel):
    input: str

def generate_animated_gif(user_prompt):
    # Assuming the output from the pipeline is already fetched
    output = pipe_(prompt=user_prompt, guidance_scale=1.0, num_inference_steps=step)

    # If output.frames is a list of lists of PIL Images
    if not output.frames or not output.frames[0]:
        raise ValueError("No frames generated")

    # Flatten the list if it's nested and filter out any non-PIL Image items
    frames = [img for sublist in output.frames for img in sublist if isinstance(img, Image.Image)]

    if not frames:
        raise ValueError("No valid frames were found in the output")

    # Save frames as GIF
    gif_file = BytesIO()
    frames[0].save(gif_file, format='GIF', save_all=True, append_images=frames[1:], loop=0, duration=100)
    gif_file.seek(0)
    gif_base64 = base64.b64encode(gif_file.read()).decode('utf-8')

    return gif_base64

@app.post("/api/generate_gif")
async def api_generate_audio(item: animate):
    gif = generate_animated_gif(item.input)
    return {"output": gif}

# **Music**

In [11]:
from transformers import AutoProcessor, MusicgenForConditionalGeneration
class music(BaseModel):
  input: str

processor = AutoProcessor.from_pretrained("facebook/musicgen-small")
model = MusicgenForConditionalGeneration.from_pretrained("facebook/musicgen-small")

@app.post("/api/generate_music")
async def generate_music(item: music):
        inputs = processor(
            text=[item.input],
            padding=True,
            return_tensors="pt",
        )

        # Generate audio values
        audio_values = model.generate(**inputs, max_new_tokens=256)

        audio_file = BytesIO()
        torchaudio.save(audio_file, torch.tensor(audio_values[0]), 24000, format="wav")
        audio_file.seek(0)
        audio_base64 = base64.b64encode(audio_file.read()).decode('utf-8')

        return {"output": audio_base64}

/usr/local/lib/python3.10/dist-packages/torch/nn/utils/weight_norm.py:28: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")
/usr/local/lib/python3.10/dist-packages/transformers/models/encodec/modeling_encodec.py:120: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.register_buffer("padding_total", torch.tensor(kernel_size - stride, dtype=torch.int64), persistent=False)


In [12]:
!ngrok config add-authtoken 2jRZ4PPjASkmxNIA5HVzbbpOIZt_3Db1d65vWC1JRZzYzN9ta

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [13]:
ngrok_tunnel = ngrok.connect(8000)
print('Public URL:', ngrok_tunnel.public_url)
nest_asyncio.apply()
uvicorn.run(app, port=8000)

Public URL: https://4626-34-125-42-159.ngrok-free.app


INFO:     Started server process [19718]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


  0%|          | 0/28 [00:00<?, ?it/s]

INFO:     129.78.56.131:0 - "POST /api/text_to_image HTTP/1.1" 200 OK


`torch.nn.functional.scaled_dot_product_attention` does not support having an empty attention mask. Falling back to the manual attention implementation. This warning can be removed using the argument `attn_implementation="eager"` when loading the model.Note that this probably happens because `guidance_scale>1` or because you used `get_unconditional_inputs`. See https://github.com/huggingface/transformers/issues/31189 for more information.


...

  0%|          | 0/4 [00:00<?, ?it/s]

INFO:     129.78.56.131:0 - "POST /api/generate_gif HTTP/1.1" 200 OK


text:   0%|          | 0/384(max) [00:00, ?it/s]We detected that you are passing `past_key_values` as a tuple and this is deprecated and will be removed in v4.43. Please use an appropriate `Cache` class (https://huggingface.co/docs/transformers/v4.41.3/en/internal/generation_utils#transformers.Cache)
text:   3%|▎         | 13/384(max) [00:01, 12.12it/s]
code:   5%|▌         | 106/2048(max) [00:03, 29.77it/s]


INFO:     129.78.56.131:0 - "POST /api/generate_audio HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [19718]
